In [1]:
import scapy

In [2]:
# Specify the PCAP trace file

pcap_trace = "2018-05-09-192.168.100.103.pcap"

In [7]:
# Using Scapy to read a PCAP file

from scapy.utils import PcapReader
from scapy.layers.inet import IP, TCP, UDP

# Metrics to collect about a specific PCAP trace
packet_count = 0
total_bytes = 0
protocol_counts = {"TCP": 0, "UDP": 0, "ICMP": 0, "OTHER": 0}
unique_src_ips = set()
unique_dst_ips = set()

with PcapReader(pcap_trace) as pcap:
    for pkt in pcap:
        packet_count += 1
        total_bytes += len(pkt)

        # IP-level info
        if IP in pkt:
            ip = pkt[IP]
            unique_src_ips.add(ip.src)
            unique_dst_ips.add(ip.dst)

            # Protocol counting
            if TCP in pkt:
                protocol_counts["TCP"] += 1
            elif UDP in pkt:
                protocol_counts["UDP"] += 1
            elif ip.proto == 1:
                protocol_counts["ICMP"] += 1
            else:
                protocol_counts["OTHER"] += 1

In [9]:
# Total metrics about packets

print("Total packets:", packet_count)
print("Total bytes:", total_bytes)
print("Average packet size:", total_bytes / packet_count)

print("\nProtocol counts:", protocol_counts)
print("Unique source IPs:", len(unique_src_ips))
print("Unique destination IPs:", len(unique_dst_ips))

Total packets: 1686291
Total bytes: 118947551
Average packet size: 70.53797416934562

Protocol counts: {'TCP': 1216851, 'UDP': 415000, 'ICMP': 21191, 'OTHER': 0}
Unique source IPs: 19630
Unique destination IPs: 597125
